<a href="https://colab.research.google.com/github/ShinAsakawa/2015corona/blob/master/notebooks/2021cnps_ccap3_tSNE_distance_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TLPA 図版の単語を word2vec でベクトル化し tSNE で距離を求めるデモ

- date: 2021_0830
- filename: 2021cnps_ccap3_tSNE_distance.ipynb
- author: 浅川伸一 <asakawa@ieee.org>
- note: 2021cnps 配布用
- License: MIT License


In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

In [ ]:
# 形態素分析ライブラリ MeCab と 辞書 mecab-ipadic-NEologd のインストール 
# reference: https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n
!pip install mecab-python3

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
# 動作確認
import MeCab
neologd_path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = MeCab.Tagger(neologd_path +' -Oyomi')
print(m.parse('鬼滅の刃'))

In [ ]:
#訓練済 word2vec ファイルの取得
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb" -O 2021_05jawiki_hid128_win10_neg10_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M" -O 2021_05jawiki_hid128_win10_neg10_sgns.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s" -O 2021_05jawiki_hid200_win20_neg20_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7" -O 2021_05jawiki_hid200_win20_neg20_sgns.bin.gz && rm -rf /tmp/cookies.txt

#旧版の訓練済 word2vec データの取得
!wget http://www.cis.twcu.ac.jp/~asakawa/2017jpa/2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz

In [ ]:
#直上のセルで取得したファイルの読み込み
#word2vec データ処理のため gensim を使う
import os
import sys
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

print('# word2vec データの読み込み')
print('# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...', end="")
# ファイルの所在に応じて変更してください
w2v_base = '.'
#w2v_file='2021_05jawiki_hid128_win10_neg10_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid128_win10_neg10_sgns.bin.gz'
#w2v_file='2021_05jawiki_hid200_win20_neg20_cbow.bin.gz'
w2v_file='2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid200_win20_neg20_sgns.bin.gz'
w2v_file = os.path.join(w2v_base, w2v_file)
w2v = KeyedVectors.load_word2vec_format(w2v_file, 
                                        encoding='utf-8', 
                                        unicode_errors='replace',
                                        binary=True) 

In [ ]:
# 散布図中に日本語を表示するための準備
!pip install japanize_matplotlib

import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

In [ ]:
# TLPA のデータ
tlpa_labels = ['バス', '緑', '桜', 'のり巻き', '五重塔', 'コップ', 'ごぼう', '土踏まず', '風呂', 'ヒトデ', 'ハム', '兎', 'ロープウエイ', '学校', 'ちりとり', '縁側', '歯', 'ネギ', 'あじさい', '灰色', '天井', '鍵', '肌色', 'ワニ', '電車', '顔', '松', 'ガードレール', '柿', 'ちまき', '信号', 'すすき', 'じょうろ', 'コンセント', '天ぷら', '中指', 'ヨット', 'ピンク', 'ふくろう', 'みかん', '柱', '角砂糖', '犬', 'かご', 'バラ', '鍋', 'まぶた', 'くるみ', '黒', 'デパート', 'カーネーション', '城', '蟻', '豆腐', 'ドライバー', '紺', '階段', '戦車', '人参', '背中', '鏡餅', 'スプーン', '朝顔', '金', '足', 'ふすま', '蛇', 'レモン', '公園', '乳母車', '床', '藤', 'ピンセット', 'トラック', '苺', '黄土色', '銭湯', 'ナマズ', 'そば', 'お腹', 'オレンジ', 'バター', '工場', '鳩', '電卓', '喉仏', 'チューリップ', '白菜', 'トラクター', '廊下', 'パトカー', '押入れ', '鉛筆', '目尻', '芋', '吊り橋', '赤', 'かき氷', '豹', 'サボテン', 'ピラミッド', 'サイ', '目', 'ひまわり', 'はたき', '刺し身', '玄関', 'トマト', '黄緑', '三輪車', '鶏', 'つむじ', 'アスパラガス', 'ドア', '銀色', 'すりこ木', 'ウイスキー', '梅', 'タクシー', '動物園', '床の間', '焦げ茶', 'ぶどう', '飴', '毛虫', 'アイロン', '寺', 'そり', 'ひょうたん', '首', '消しゴム', '頬', 'いちょう', '駅', 'ギョウザ', '牛', 'びわ', '飛行機', '畳', '白', '竹', 'ペリカン', '紫', '手すり', '口', '大根', '風車', '鋏', '潜水艦', 'ステーキ', 'マッチ', '二階', '落花生', '御飯', '自転車', '歩道橋', '鯨', '茶色', '菖蒲', 'ふくらはぎ', '桃', 'たいやき', '道路', '靴べら', '水色', '壁', 'たんぽぽ', 'いかだ', '山羊', '鼻', '海老', '台所', 'オートバイ', 'かぶ', '柳', 'しゃもじ', 'まんじゅう', 'かかと', '薄紫', '家', 'おせち料理', '青', '傘', 'つくし', 'りんご', '馬車', '線路', 'タツノオトシゴ', '耳', '便所', '蓮根', '猫', '黄色', 'へそ', '街灯', '障子', '酒', '船', '安全ピン', 'もみじ']
tlpa_fam = ['高', '高', '高', '低', '低', '高', '低', '低', '高', '低', '高', '高', '低', '高', '低', '低', '高', '高', '低', '低', '高', '高', '低', '低', '高', '高', '高', '低', '低', '低', '高', '低', '低', '低', '高', '低', '高', '高', '低', '高', '低', '低', '高', '低', '高', '高', '低', '低', '高', '高', '低', '低', '高', '高', '低', '低', '高', '低', '高', '高', '低', '高', '高', '低', '高', '低', '高', '低', '高', '低', '高', '低', '低', '高', '高', '低', '低', '低', '高', '高', '高', '高', '高', '高', '低', '低', '高', '低', '低', '低', '高', '高', '高', '低', '高', '低', '高', '低', '低', '低', '低', '低', '高', '高', '低', '高', '高', '高', '低', '低', '高', '低', '低', '高', '低', '低', '低', '高', '高', '高', '低', '低', '高', '高', '低', '高', '高', '低', '低', '高', '高', '低', '低', '高', '低', '高', '低', '高', '低', '高', '高', '低', '高', '低', '高', '高', '低', '高', '低', '低', '高', '低', '低', '高', '高', '低', '高', '高', '低', '低', '高', '低', '高', '低', '低', '高', '高', '低', '低', '高', '高', '高', '高', '低', '低', '低', '高', '低', '低', '高', '低', '高', '高', '低', '高', '低', '低', '低', '高', '高', '低', '高', '高', '低', '低', '低', '高', '高', '低', '高']
tlpa_cat = ['乗り物', '色', '植物', '加工食品', '建造物', '道具', '野菜果物', '身体部位', '屋内部位', '動物', '加工食品', '動物', '乗り物', '建造物', '道具', '屋内部位', '身体部位', '野菜果物', '植物', '色', '屋内部位', '道具', '色', '動物', '乗り物', '身体部位', '植物', '建造物', '野菜果物', '加工食品', '建造物', '植物', '道具', '屋内部位', '加工食品', '身体部位', '乗り物', '色', '動物', '野菜果物', '屋内部位', '加工食品', '動物', '乗り物', '植物', '道具', '身体部位', '野菜果物', '色', '建造物', '植物', '建造物', '動物', '加工食品', '道具', '色', '屋内部位', '乗り物', '野菜果物', '身体部位', '加工食品', '道具', '植物', '色', '身体部位', '屋内部位', '動物', '野菜果物', '建造物', '乗り物', '屋内部位', '植物', '道具', '乗り物', '野菜果物', '色', '建造物', '動物', '加工食品', '身体部位', '色', '加工食品', '建造物', '動物', '道具', '身体部位', '植物', '野菜果物', '乗り物', '屋内部位', '乗り物', '屋内部位', '道具', '身体部位', '野菜果物', '建造物', '色', '加工食品', '動物', '植物', '建造物', '動物', '身体部位', '植物', '道具', '加工食品', '屋内部位', '野菜果物', '色', '乗り物', '動物', '身体部位', '野菜果物', '屋内部位', '色', '道具', '加工食品', '植物', '乗り物', '建造物', '屋内部位', '色', '野菜果物', '加工食品', '動物', '道具', '建造物', '乗り物', '植物', '身体部位', '道具', '身体部位', '植物', '建造物', '加工食品', '動物', '野菜果物', '乗り物', '屋内部位', '色', '植物', '動物', '色', '屋内部位', '身体部位', '野菜果物', '建造物', '道具', '乗り物', '加工食品', '道具', '屋内部位', '野菜果物', '加工食品', '乗り物', '建造物', '動物', '色', '植物', '身体部位', '野菜果物', '加工食品', '建造物', '道具', '色', '屋内部位', '植物', '乗り物', '動物', '身体部位', '動物', '屋内部位', '乗り物', '野菜果物', '植物', '道具', '加工食品', '身体部位', '色', '建造物', '加工食品', '色', '道具', '植物', '野菜果物', '乗り物', '建造物', '動物', '身体部位', '屋内部位', '野菜果物', '動物', '色', '身体部位', '建造物', '屋内部位', '加工食品', '乗り物', '道具', '植物']


In [ ]:
import sys
import numpy as np
"""
- source: https://lvdmaaten.github.io/tsne/
- オリジナルの tSNE python 実装を python 3 系で呼び出せるように変更したバージョン
- date: 2021_0510
- author: 浅川伸一

```python
import numpy as np
import tsne

X = np.random.random(100, 30)
result = tsne.tsne(X)
```
"""
#  tsne.py
#
# Implementation of t-SNE in Python. The implementation was tested on Python 2.7.10, and it requires a working
# installation of NumPy. The implementation comes with an example on the MNIST dataset. In order to plot the
# results of this example, a working installation of matplotlib is required.
#
# The example can be run by executing: `ipython tsne.py`
#
#
#  Created by Laurens van der Maaten on 20-12-08.
#  Copyright (c) 2008 Tilburg University. All rights reserved.

#import numpy as Math
#import pylab as Plot

def Hbeta(D = np.array([]), beta = 1.0):
    """Compute the perplexity and the P-row for a specific value of the precision of a Gaussian distribution."""

    # Compute P-row and corresponding perplexity
    P = np.exp(-D.copy() * beta)
    sumP = sum(P) if sum(P) > 1e-12 else 1e-12    # to avoid division by zero ここだけ加えた。直下行が division by zero error にならないように 
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P


def x2p(X = np.array([]), tol=1e-5, perplexity=30.0):
    """Performs a binary search to get P-values in such a way that each conditional Gaussian has the same perplexity."""

    # Initialize some variables
    #print("Computing pairwise distances...")
    (n, d) = X.shape
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)

    # Loop over all datapoints
    for i in range(n):

        # Print progress
        #if i % 500 == 0:
        #    print("Computing P-values for point ", i, " of ", n, "...")

        # Compute the Gaussian kernel and entropy for the current precision
        betamin = -np.inf
        betamax =  np.inf
        Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        (H, thisP) = Hbeta(Di, beta[i])

        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > tol and tries < 50:

            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2
                else:
                    beta[i] = (beta[i] + betamax) / 2;
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2
                else:
                    beta[i] = (beta[i] + betamin) / 2;

            # Recompute the values
            (H, thisP) = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries = tries + 1

        # Set the final row of P
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

    # Return final P-matrix
    sigma = np.mean(np.sqrt(1 / beta))
    print(f'Mean value of sigma: {sigma:.3f}')
    return P


def pca(X = np.array([]), no_dims = 50):
    """Runs PCA on the NxD array X in order to reduce its dimensionality to no_dims dimensions."""

    #print("Preprocessing the data using PCA...")
    (n, d) = X.shape
    X = X - np.tile(np.mean(X, 0), (n, 1))
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(X, M[:,0:no_dims])
    return Y


def tsne(X = np.array([]), no_dims=2, initial_dims=50, perplexity=30.0):
    """
    Runs t-SNE on the dataset in the NxD array X to reduce its dimensionality to no_dims dimensions.
    The syntaxis of the function is Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array.
    """

    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array X should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1

    # Initialize variables
    X = pca(X, initial_dims).real
    (n, d) = X.shape
    max_iter = 1000
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 500
    min_gain = 0.01
    Y = np.random.randn(n, no_dims)
    dY = np.zeros((n, no_dims))
    iY = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))

    # Compute P-values
    P = x2p(X, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = P * 4   # early exaggeration
    P = np.maximum(P, 1e-12)
    #P = np.maximum(P, 1e-5)

    interval = int(max_iter >> 2)
    # Run iterations
    for iter in range(max_iter):
        # Compute pairwise affinities
        sum_Y = np.sum(np.square(Y), 1)
        num = 1 / (1 + np.add(np.add(-2 * np.dot(Y, Y.T), sum_Y).T, sum_Y))
        num[range(n), range(n)] = 0
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)
        #Q = np.maximum(Q, 1e-5)

        # Compute gradient
        PQ = P - Q;
        for i in range(n):
            dY[i,:] = np.sum(np.tile(PQ[:,i] * num[:,i], (no_dims, 1)).T * (Y[i,:] - Y), 0)

        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dY > 0) != (iY > 0)) + (gains * 0.8) * ((dY > 0) == (iY > 0))
        gains[gains < min_gain] = min_gain
        iY = momentum * iY - eta * (gains * dY)
        Y = Y + iY
        Y = Y - np.tile(np.mean(Y, 0), (n, 1))

        # Compute current value of cost function
        #if (iter + 1) % 10 == 0:
        #if (iter + 1) % interval == 0:
        #    C = np.sum(P * np.log(P / Q))
        #    print(f"Iteration {(iter + 1):<5d}: error is {C:.3f}")

        # Stop lying about P-values
        if iter == 100:
            P = P / 4;

    # Return solution
    return Y;


# if __name__ == "__main__":
# 	print("Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset.")
# 	print("Running example on 2,500 MNIST digits...")
# 	X = np.loadtxt("mnist2500_X.txt");
# 	labels = np.loadtxt("mnist2500_labels.txt");
# 	Y = tsne(X, 2, 50, 20.0);
# 	Plot.scatter(Y[:,0], Y[:,1], 20, labels);
# 	Plot.show();


In [ ]:
tlpa_cats = list(set(tlpa_cat))
tlpa_colors = [tlpa_cats.index(c) for c in tlpa_cat]

def draw_tSNE_plot(data, 
                   colors=tlpa_colors,
                   labels=tlpa_labels, 
                   fontsize=16, 
                   figsize=(16,16),
                   xmax=None, xmin=None,
                   ymax=None, ymin=None,
                   save_figname=None,
                   grid = True,
                   auto_lim = True,
                  ):
    """tSNE のプロットを描画する関数
    
    引数:
    data: np.array[N,2]
        tSNE の結果
    colors: list[N] 
        各項目の色を指定する数字 N 個
    labels: list[str]
        散布図中に表示する項目名のリスト
    figsize: タプル
        縦横のサイズ。単位はインチ。だが昔と違ってディスプレイサイズがまちまちなので目安でしか無い
    xmax, xmin, ymax, ymin: int
        図の最大値と最小値を指定する。
        指定しなければ自動計算する
    save_figname: str
        保存するファイル名 pdf ファイルとして保存するなら .pdf 拡張子をつける
    grid: Boolean
        図中にグリッドを表示するか否か。デフォルトは True
    auto_lim: Boolean
        最大値最小値を自動計算するか否か
        xmax, xmin, ymax, ymin が指定されていれば自動的に False になる
    """

    fig = plt.figure(figsize=figsize)
    axe = fig.add_subplot(1,1,1)

    if xmax == None: xmax = (X[:,0]).max(); auto_lim = False
    if xmin == None: xmin = (X[:,0]).min(); auto_lim = False
    if ymax == None: ymax = (X[:,1]).max(); auto_lim = False
    if ymin == None: ymin = (X[:,1]).min(); auto_lim = False
    if auto_lim: 
        axe.set_xlim(xmin, xmax); axe.set_ylim(ymin, ymax)

    if grid:
        axe.grid()

    axe.scatter(data[:,0], data[:,1], 120, colors)
    for i, label in enumerate(labels):
        axe.annotate(label, (data[i,0], data[i,1]), fontsize=fontsize)

    if save_figname != None:
        plt.savefig(save_figname)
        
    return fig, axe

In [ ]:
# seed = 0
# np.random.seed(seed)
# X = np.array([w2v[word] for word in tlpa_labels], dtype=np.float)
# tlpa_cats = list(set(tlpa_cat))
# tlpa_colors = [tlpa_cats.index(c) for c in tlpa_cat]
# tlpa_results = tsne(X, perplexity=30.0)
# f, axe = draw_tSNE_plot(tlpa_results, labels=tlpa_labels, colors=tlpa_colors)

In [ ]:
# 大門先生のデータ，天ぷら ---> たくあん と言う意味性錯語を検証
def compare_distances_tSNE_word2vec(t_word='天ぷら', r_word='たくあん', 
                                    tlpa_labels=tlpa_labels, 
                                    seed=0, 
                                    perplexity=30.0, 
                                    save_fig=False,
                                    draw_fig=True
                                   ):
    
    # TLPA の単語ベクトルを X に代入
    X = np.array([w2v[word] for word in tlpa_labels], dtype=np.float)
    r_vec = [w2v[r_word]]  # 反応語の単語ベクトルを取得し r_vec に代入
    
    X_ = np.concatenate((X, r_vec), axis=0)  # X と r_vec を併せて新しい X_ を作成する
    # 上の X_ に合わせてラベルデータ tlpa_labels_ を作成
    tlpa_labels_ = ['バス', '緑', '桜', 'のり巻き', '五重塔', 'コップ', 'ごぼう', '土踏まず', '風呂', 'ヒトデ', \
                    'ハム', '兎', 'ロープウエイ', '学校', 'ちりとり', '縁側', '歯', 'ネギ', 'あじさい', '灰色', \
                    '天井', '鍵', '肌色', 'ワニ', '電車', '顔', '松', 'ガードレール', '柿', 'ちまき', '信号', \
                    'すすき', 'じょうろ', 'コンセント', '天ぷら', '中指', 'ヨット', 'ピンク', 'ふくろう', 'みかん', \
                    '柱', '角砂糖', '犬', 'かご', 'バラ', '鍋', 'まぶた', 'くるみ', '黒', 'デパート', 'カーネーション', \
                    '城', '蟻', '豆腐', 'ドライバー', '紺', '階段', '戦車', '人参', '背中', '鏡餅', 'スプーン', \
                    '朝顔', '金', '足', 'ふすま', '蛇', 'レモン', '公園', '乳母車', '床', '藤', 'ピンセット', \
                    'トラック', '苺', '黄土色', '銭湯', 'ナマズ', 'そば', 'お腹', 'オレンジ', 'バター', '工場', \
                    '鳩', '電卓', '喉仏', 'チューリップ', '白菜', 'トラクター', '廊下', 'パトカー', '押入れ', \
                    '鉛筆', '目尻', '芋', '吊り橋', '赤', 'かき氷', '豹', 'サボテン', 'ピラミッド', 'サイ', '目', \
                    'ひまわり', 'はたき', '刺し身', '玄関', 'トマト', '黄緑', '三輪車', '鶏', 'つむじ', 'アスパラガス', \
                    'ドア', '銀色', 'すりこ木', 'ウイスキー', '梅', 'タクシー', '動物園', '床の間', '焦げ茶', 'ぶどう', \
                    '飴', '毛虫', 'アイロン', '寺', 'そり', 'ひょうたん', '首', '消しゴム', '頬', 'いちょう', '駅', \
                    'ギョウザ', '牛', 'びわ', '飛行機', '畳', '白', '竹', 'ペリカン', '紫', '手すり', '口', '大根', \
                    '風車', '鋏', '潜水艦', 'ステーキ', 'マッチ', '二階', '落花生', '御飯', '自転車', '歩道橋', '鯨', \
                    '茶色', '菖蒲', 'ふくらはぎ', '桃', 'たいやき', '道路', '靴べら', '水色', '壁', 'たんぽぽ', \
                    'いかだ', '山羊', '鼻', '海老', '台所', 'オートバイ', 'かぶ', '柳', 'しゃもじ', 'まんじゅう', \
                    'かかと', '薄紫', '家', 'おせち料理', '青', '傘', 'つくし', 'りんご', '馬車', '線路', \
                    'タツノオトシゴ', '耳', '便所', '蓮根', '猫', '黄色', 'へそ', '街灯', '障子', '酒', '船', \
                    '安全ピン', 'もみじ', r_word]
    # 反応語を最後に入れたので，その語の色を 11 番目の色として設定
    tlpa_colors_ = [8, 7, 5, 0, 2, 4, 9, 6, 3, 1, 0, 1, 8, 2, 4, 3, 6, 9, 5, 7, 3, 4, 7, 1, 8, 6, 5, 2, 9, 0, 2, 5, 4, 3, 0, 6, 8, 7, 1, 9, 3, 0, 1, 8, 5, 4, 6, 9, 7, 2, 5, 2, 1, 0, 4, 7, 3, 8, 9, 6, 0, 4, 5, 7, 6, 3, 1, 9, 2, 8, 3, 5, 4, 8, 9, 7, 2, 1, 0, 6, 7, 0, 2, 1, 4, 6, 5, 9, 8, 3, 8, 3, 4, 6, 9, 2, 7, 0, 1, 5, 2, 1, 6, 5, 4, 0, 3, 9, 7, 8, 1, 6, 9, 3, 7, 4, 0, 5, 8, 2, 3, 7, 9, 0, 1, 4, 2, 8, 5, 6, 4, 6, 5, 2, 0, 1, 9, 8, 3, 7, 5, 1, 7, 3, 6, 9, 2, 4, 8, 0, 4, 3, 9, 0, 8, 2, 1, 7, 5, 6, 9, 0, 2, 4, 7, 3, 5, 8, 1, 6, 1, 3, 8, 9, 5, 4, 0, 6, 7, 2, 0, 7, 4, 5, 9, 8, 2, 1, 6, 3, 9, 1, 7, 6, 2, 3, 0, 8, 4, 5, 10]

    t_num = tlpa_labels_.index(t_word)
    r_num = tlpa_labels_.index(r_word)
    tlpa_colors_[t_num] = 10  # 図を見やすくするため，ターゲット語の色を表出語の色と同じ 11 番目の色に設定

    np.random.seed(seed)  # 乱数の種を設定
    tlpa_results_ = tsne(X_, perplexity=perplexity)  # tSNE の実行
    if draw_fig:
        f, axe = draw_tSNE_plot(tlpa_results_, labels=tlpa_labels_, colors=tlpa_colors_)
    if save_fig:
        save_fname = '2021_0825'+t_word+'_'+r_word+'.pdf'
        plt.savefig(save_fname)
    plt.show()
    a = tlpa_results_[t_num]  # ターゲット語の tSNE 座標を取得して a に代入
    b = tlpa_results_[r_num]  # 表出語の tSNE 座標を取得して b に代入
    #print(f'{t_word}: {a} {r_word}: {b}')  # 結果を表示
    tsne_dist = np.linalg.norm(a - b)      # ターゲット語と表出語のユークリッド距離を計算し tsne_dist に代入
    w2v_dist = w2v.distance(t_word, r_word)
    #print(f'{t_word} と {r_word} との tSNE 上での距離: {dist:.3f}',
    #      f'word2vec 上での距離: {w2v_dist:.3f}')
    return tsne_dist, w2v_dist



# #大門先生のデータ，各リストの要素はタプルで，タプルの先頭が刺激語，2番目が反応
# daimon_results = [ ('あじさい', 'フラワー'), ('ちまき','ふきのとう'), 
#                   ('天ぷら','たくあん'), ('角砂糖','ストロー'), 
#                   ('角砂糖', 'フォーク'), ('鍋','やかん'),
#                   ('城','やぐら'), ('廊下','戸締り'), 
#                   ('安全ピン','ピンセット')  #これだけは形式性錯語
#                  ]

# for pair in daimon_results:
#     tsne_dist, w2v_dist = compare_distances_tSNE_word2vec(pair[0], pair[1], save_fig=False, draw_fig=False)
#     print(f'{pair[0]} と {pair[1]} との tSNE 上での距離: {tsne_dist:.3f}, w2v 上での距離(1-similarity):{w2v_dist:.3f}')
 


In [ ]:
w_target = input('TLPA 検査項目 200 語のうち一語を入力してください')
w_resp   = input('対応する産出された語を入力してください') 
tsne_dist, w2v_dist = compare_distances_tSNE_word2vec(w_target, w_resp, save_fig=False, draw_fig=False)
print(f'{w_target} と {w_resp} との tSNE 上での距離: {tsne_dist:.3f}, w2v 上での距離(非類似度):{w2v_dist:.3f}')